In [1]:
import requests

In [15]:
standings_url = "https://fbref.com/en/comps/4/stats/WCQ----CONMEBOL-M-Stats"

In [16]:
data = requests.get(standings_url)

In [17]:
from bs4 import BeautifulSoup

In [18]:
soup = BeautifulSoup(data.text)
standings_table = soup.select('table.stats_table')[0]
links = standings_table.find_all('a')
links = [l.get("href") for l in links]
links = [l for l in links if '/squads/' in l]

In [19]:
team_urls = [f"https://fbref.com{l}" for l in links]

In [20]:
data = requests.get(team_urls[0])

In [21]:
import pandas as pd
matches = pd.read_html(data.text, match="Scores & Fixtures")[0]

In [22]:
soup = BeautifulSoup(data.text)
links = soup.find_all('a')
links = [l.get("href") for l in links]
links = [l for l in links if l and 'all_comps/shooting/' in l]

In [23]:
data = requests.get(f"https://fbref.com{links[0]}")

In [24]:
shooting = pd.read_html(data.text, match="Shooting")[0]

In [25]:
shooting.head()

For Argentina                                                          \
           Date   Time Comp               Round  Day Venue Result GF GA   
0    2020-10-08  21:30  WCQ  WCQ — CONMEBOL (M)  Thu  Home      W  1  0   
1    2020-10-13  16:00  WCQ  WCQ — CONMEBOL (M)  Tue  Away      W  2  1   
2    2020-11-12  21:00  WCQ  WCQ — CONMEBOL (M)  Thu  Home      D  1  1   
3    2020-11-17  19:30  WCQ  WCQ — CONMEBOL (M)  Tue  Away      W  2  0   
4    2021-06-03  21:00  WCQ  WCQ — CONMEBOL (M)  Thu  Home      D  1  1   

               Standard                                             \
      Opponent      Gls    Sh  SoT  SoT%  G/Sh G/SoT Dist PK PKatt   
0   ec Ecuador        1   8.0  2.0  25.0  0.00  0.00  NaN  1     1   
1   bo Bolivia        2  17.0  3.0  17.6  0.12  0.67  NaN  0     0   
2  py Paraguay        1  15.0  5.0  33.3  0.07  0.20  NaN  0     0   
3      pe Peru        2  13.0  3.0  23.1  0.15  0.67  NaN  0     0   
4     cl Chile        1  16.0  7.0  43.8  0.00  0.00  NaN  1     1   

  Unnamed: 19_level_0  
         Match Report  
0        Match Report  
1        Match Report  
2        Match Report  
3        Match Report  
4        Match Report

In [26]:
shooting.columns = shooting.columns.droplevel()

In [30]:
# team_data = matches.merge(shooting[["Date", "Sh", "SoT", "Dist", "G-PK", "PK", "PKatt"]], on="Date")



In [32]:
# team_data.head()

In [33]:
years = list(range(2022, 2020, -1))
all_matches = []

In [34]:
standings_url = "https://fbref.com/en/comps/1/FIFA-World-Cup-Stats"

In [36]:
import time
for year in years:
    data = requests.get(standings_url)
    soup = BeautifulSoup(data.text)
    standings_table = soup.select('table.stats_table')[0]

    links = [l.get("href") for l in standings_table.find_all('a')]
    links = [l for l in links if '/squads/' in l]
    team_urls = [f"https://fbref.com{l}" for l in links]
    
    previous_season = soup.select("a.prev")[0].get("href")
    standings_url = f"https://fbref.com{previous_season}"
    
    for team_url in team_urls:
        team_name = team_url.split("/")[-1].replace("-Stats", "").replace("-", " ")
        data = requests.get(team_url)
        matches = pd.read_html(data.text, match="Scores & Fixtures")[0]
        soup = BeautifulSoup(data.text)
        links = [l.get("href") for l in soup.find_all('a')]
        links = [l for l in links if l and 'all_comps/shooting/' in l]
        data = requests.get(f"https://fbref.com{links[0]}")
        shooting = pd.read_html(data.text, match="Shooting")[0]
        shooting.columns = shooting.columns.droplevel()
        try:
            team_data = matches.merge(shooting[["Date", "Sh", "SoT", "Dist", "PK", "PKatt"]], on="Date")
        except ValueError:
            continue
        team_data = team_data[team_data["Comp"] == "Premier League"]
        
        team_data["Season"] = year
        team_data["Team"] = team_name
        all_matches.append(team_data)
        time.sleep(1)

ValueError: No tables found

In [ ]:
len(all_matches)

In [ ]:
match_df = pd.concat(all_matches)

In [ ]:
match_df.columns = [c.lower() for c in match_df.columns]

In [ ]:
match_df

In [ ]:
match_df.to_csv("matches.csv")